#  Standard Star Tutorial
Create a simulated observation of a standard A0 star

In [1]:
from pyechelle.simulator import Simulator
from pyechelle.telescope import Telescope
from pyechelle.sources import Phoenix
from pyechelle.sources import CSV
from pyechelle.spectrograph import ZEMAX

import numpy as np

from astropy.io import fits
import astropy.units as u
import scipy.io

### Setup paths and file names
Change the output path.

In [2]:
output_path = '/home/aidan/data/hispec_simulations/'

sci_fiber_model = 'HISPEC_HK_SCI'
lfc_fiber_model = 'HISPEC_HK_LFC'
sky_fiber_model = 'HISPEC_HK_SKY_BKGD'
dark_fiber_model = 'HISPEC_HK_DARK'

### General observing parameters

In [3]:
t_exp = 1.0 # s
tele = Telescope(9.96,0)

### LFC Fiber

In [4]:
lfc_15_27_path = '/home/aidan/code/HISPECEchelleSim/lfc/20220513_1.5-3.4um/disp2_2.145_disp3_-0.05_ptamp_4.5A_1.5-2.7um.mat'
lfc_09_17_path = '/home/aidan/code/HISPECEchelleSim/lfc/20220513_0.6-1.7um/disp2_2.145_disp3_-0.05_ptamp_4.5A_0.9-1.7um_highres.mat'
lfc_15_27 = scipy.io.loadmat(lfc_15_27_path) 
lfc_09_17 = scipy.io.loadmat(lfc_09_17_path) 

lfc_15_27_power = np.power(10, lfc_15_27['OSAPower'] / 10) * 1e4 #* u.erg/u.s/u.cm**2 
lfc_15_27_wvs = (lfc_15_27['OSAWavelength'] * u.nm).to(u.micron)
lfc_09_17_power = np.power(10, lfc_09_17['OSAPower'] / 10) * 1e4 #* u.erg/u.s/u.cm**2 
lfc_09_17_wvs = (lfc_09_17['OSAWavelength'] * u.nm).to(u.micron)

idx_17 = np.argmin(abs(lfc_15_27_wvs.value - 1.7))
lfc_power = np.concatenate((lfc_09_17_power.flatten(), lfc_15_27_power.flatten()[idx_17+1:]), axis=0)* u.erg/u.s/u.cm**2 
lfc_wvs = np.concatenate((lfc_09_17_wvs.flatten(), lfc_15_27_wvs.flatten()[idx_17+1:]))



# converting the power to ph/s/cm2/cm, should be units for input to pyechelle
lfc_spec = lfc_power.to(u.ph/u.s/u.cm**2,equivalencies=u.spectral_density(lfc_wvs))

In [5]:
# write lfc spec to csv
lfc = np.column_stack((lfc_wvs.value * 1e4, lfc_spec.value/1e10))
np.savetxt(output_path+'lfc_spec.csv', lfc, delimiter=',')

Run the simulation for the lfc fiber

In [6]:
lfc_sim = Simulator(ZEMAX(lfc_fiber_model))
lfc_sim.set_ccd(1) # always leave as 1
lfc_sim.set_fibers(1) # leave as 1 since the fibers are in separate models
lfc_sim.set_sources(CSV(output_path+'lfc_spec.csv', flux_in_photons=True)) 
lfc_sim.set_exposure_time(t_exp) # s
lfc_sim.set_output(output_path+lfc_fiber_model+'_texp'+str(t_exp)+'.fits', overwrite=True)
lfc_sim.set_telescope(tele) 
lfc_sim.set_orders([*range(59, 98)]) # a list of all expected orders 59 to 97

lfc_sim.max_cpu = 1 # set number of cpu to use
lfc_sim.run()

filepath=PosixPath('/home/aidan/data/hispec_simulations/lfc_spec.csv'), <class 'pathlib.PosixPath'>
Order  59:     2441.8 -  2477.6 nm.     Number of photons: 356331
Order  60:     2401.1 -  2436.3 nm.     Number of photons: 684786
Order  61:     2361.8 -  2396.3 nm.     Number of photons: 849522
Order  62:     2323.6 -  2357.8 nm.     Number of photons: 928728
Order  63:     2286.7 -  2320.3 nm.     Number of photons: 1472802
Order  64:     2251.1 -  2284.0 nm.     Number of photons: 2710328
Order  65:     2216.4 -  2248.9 nm.     Number of photons: 4501507
Order  66:     2182.8 -  2214.8 nm.     Number of photons: 6865426
Order  67:     2150.2 -  2181.8 nm.     Number of photons: 10002552
Order  68:     2118.6 -  2149.8 nm.     Number of photons: 13210591
Order  69:     2087.9 -  2118.5 nm.     Number of photons: 17009190
Order  70:     2058.1 -  2088.3 nm.     Number of photons: 21135855
Order  71:     2029.1 -  2058.9 nm.     Number of photons: 25044240
Order  72:     2000.9 -  203

INFO:Simulator:Finish up simulation and save...
INFO:Simulator:Total time for simulation: 215.317s.
INFO:Simulator:Total simulated photons: 586011194


586011194